# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = '/home/carnd/train.p'
validation_file = '/home/carnd/valid.p'
testing_file = '/home/carnd/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [2]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

import numpy as np

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = (len(X_train[0]), len(X_train[0][0]))
image_channels = len(X_train[0][0][0])

# TODO: How many unique classes/labels there are in the dataset.
classes = {}

for label in np.concatenate((y_train, y_valid, y_test)):
    classes[label] = True

n_classes = len(classes.keys())

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)


Number of training examples = 34799
Number of validation examples = 4410
Image data shape = (32, 32)
Number of classes = 43


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [3]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.

import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

def count_classes(labels):
    class_counts = np.array([0.0] * n_classes)
    for label in labels:
        class_counts[label] += 1

    class_counts /= len(labels)
    class_counts *= 100
    return class_counts / len(labels)


def print_classes(class_counts):
    print("Class, %\n-------------")
    
    for i in range(class_counts.shape[0]):
        print("{}, {:.3f}".format(i, class_counts[i]))
    print()
    
train_counts = count_classes(y_train)
valid_counts = count_classes(y_valid)

print_classes(np.absolute(train_counts - valid_counts))
    

Class, %
-------------
0, 0.000
1, 0.001
2, 0.001
3, 0.001
4, 0.001
5, 0.001
6, 0.000
7, 0.001
8, 0.001
9, 0.001
10, 0.001
11, 0.001
12, 0.001
13, 0.001
14, 0.000
15, 0.000
16, 0.000
17, 0.001
18, 0.001
19, 0.000
20, 0.000
21, 0.000
22, 0.000
23, 0.000
24, 0.000
25, 0.001
26, 0.000
27, 0.000
28, 0.000
29, 0.000
30, 0.000
31, 0.000
32, 0.000
33, 0.000
34, 0.000
35, 0.001
36, 0.000
37, 0.000
38, 0.001
39, 0.000
40, 0.000
41, 0.000
42, 0.000



----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [4]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

import cv2

def grayscale(images):
    gray = np.dot(np.array(images)[...,:3], [0.299, 0.587, 0.114])
    return np.reshape(gray, gray.shape + (1,))
        
def normalize(images):
    images = (images - 128.) / 128.

# Apply random rotation, translation and shear.    
def jitter(image, angle=20, shift=4, shear=1.5):
    angle = np.random.uniform(angle) - (angle/2.)
    rotation = cv2.getRotationMatrix2D((image_shape[0]/2, image_shape[1]/2), angle, 1)
    
    dx, dy = np.random.uniform(shift) - shift, np.random.uniform(shift) - shift
    shear_x, shear_y = np.random.uniform(shear) - shear, np.random.uniform(shear) - shear
    
    trans = np.array([[shear_x, 0.0, dx],[0.0, shear_y, dy]])
    
    image = cv2.warpAffine(image, rotation, ima)
    image = cv2.warpAffine(image, trans)
    return image
    
    
X_train = grayscale(X_train)
X_valid = grayscale(X_valid)
X_test = grayscale(X_test)

normalize(X_train)
normalize(X_valid)
normalize(X_test)

for i in range(len(X_train)):
    X_train[i] = jitter(X_train[i])



TypeError: Required argument 'dsize' (pos 3) not found

### Model Architecture

In [29]:
import tensorflow as tf

def Network(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    weights = [
        tf.Variable(tf.truncated_normal(shape=(3, 3, 1, 32), mean = mu, stddev = sigma)),
        tf.Variable(tf.truncated_normal(shape=(3, 3, 32, 64), mean = mu, stddev = sigma)),
        tf.Variable(tf.truncated_normal(shape=(9504, 256), mean = mu, stddev = sigma)),
        tf.Variable(tf.truncated_normal(shape=(256, 128), mean = mu, stddev = sigma)),
        tf.Variable(tf.truncated_normal(shape=(128, 43), mean = mu, stddev = sigma))
    ]
    
    biases = [
        tf.Variable(tf.zeros(32)),
        tf.Variable(tf.zeros(64)),
        tf.Variable(tf.zeros(256)),
        tf.Variable(tf.zeros(128)),
        tf.Variable(tf.zeros(43))
    ]
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 30x30x32.
    conv1   = tf.nn.conv2d(x, weights[0], strides=[1, 1, 1, 1], padding='VALID') + biases[0]
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 30x30x32. Output = 14x14x32.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. Output = 12x12x64.
    conv2   = tf.nn.conv2d(conv1, weights[1], strides=[1, 1, 1, 1], padding='VALID') + biases[1]
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 12x12x64. Output = 5x5x64.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x64. Output = 1600.
    flat0 = tf.contrib.layers.flatten(conv2)
    
    # Add connection from first conv layer to FC layers.
    flat1 = tf.contrib.layers.flatten(conv1)
    
    fc0 = tf.concat(values=[flat0, flat1], concat_dim=1)
    
    fc1   = tf.matmul(fc0, weights[2]) + biases[2]
    fc1    = tf.nn.relu(fc1)

    fc2 = tf.matmul(fc1, weights[3]) + biases[3]
    
    # Add dropout to control overfitting.
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, keep_prob)

    logits = tf.matmul(fc2, weights[4]) + biases[4]
    
    return logits, weights

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [30]:
from sklearn.utils import shuffle

x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)
keep_prob = tf.placeholder(dtype=tf.float32)

l_rate = 0.001
reg = 0.01

logits, weights = Network(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)

for w in weights:
    loss_operation = loss_operation + (reg * tf.nn.l2_loss(w))
                 
optimizer = tf.train.AdamOptimizer(learning_rate = l_rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

EPOCHS =  400
BATCH_SIZE = 256

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.6})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
    
    print("Training Accuracy = {:.3f}".format(evaluate(X_train, y_train)))
        
    saver.save(sess, './net')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.048

EPOCH 2 ...
Validation Accuracy = 0.048

EPOCH 3 ...
Validation Accuracy = 0.054

EPOCH 4 ...
Validation Accuracy = 0.096

EPOCH 5 ...
Validation Accuracy = 0.090

EPOCH 6 ...
Validation Accuracy = 0.115

EPOCH 7 ...
Validation Accuracy = 0.113

EPOCH 8 ...
Validation Accuracy = 0.115

EPOCH 9 ...
Validation Accuracy = 0.103

EPOCH 10 ...
Validation Accuracy = 0.115

EPOCH 11 ...
Validation Accuracy = 0.103

EPOCH 12 ...
Validation Accuracy = 0.125

EPOCH 13 ...
Validation Accuracy = 0.131

EPOCH 14 ...
Validation Accuracy = 0.144

EPOCH 15 ...
Validation Accuracy = 0.185

EPOCH 16 ...
Validation Accuracy = 0.182

EPOCH 17 ...
Validation Accuracy = 0.244

EPOCH 18 ...
Validation Accuracy = 0.302

EPOCH 19 ...
Validation Accuracy = 0.392

EPOCH 20 ...
Validation Accuracy = 0.464

EPOCH 21 ...
Validation Accuracy = 0.673

EPOCH 22 ...
Validation Accuracy = 0.788

EPOCH 23 ...
Validation Accuracy = 0.861

EPOCH 24 ...
Validation Accura

EPOCH 194 ...
Validation Accuracy = 0.927

EPOCH 195 ...
Validation Accuracy = 0.932

EPOCH 196 ...
Validation Accuracy = 0.931

EPOCH 197 ...
Validation Accuracy = 0.921

EPOCH 198 ...
Validation Accuracy = 0.928

EPOCH 199 ...
Validation Accuracy = 0.927

EPOCH 200 ...
Validation Accuracy = 0.926

EPOCH 201 ...
Validation Accuracy = 0.936

EPOCH 202 ...
Validation Accuracy = 0.915

EPOCH 203 ...
Validation Accuracy = 0.933

EPOCH 204 ...
Validation Accuracy = 0.923

EPOCH 205 ...
Validation Accuracy = 0.924

EPOCH 206 ...
Validation Accuracy = 0.913

EPOCH 207 ...
Validation Accuracy = 0.925

EPOCH 208 ...
Validation Accuracy = 0.923

EPOCH 209 ...
Validation Accuracy = 0.923

EPOCH 210 ...
Validation Accuracy = 0.927

EPOCH 211 ...
Validation Accuracy = 0.909

EPOCH 212 ...
Validation Accuracy = 0.932

EPOCH 213 ...
Validation Accuracy = 0.925

EPOCH 214 ...
Validation Accuracy = 0.923

EPOCH 215 ...
Validation Accuracy = 0.914

EPOCH 216 ...
Validation Accuracy = 0.921

EPOCH 217 .

EPOCH 385 ...
Validation Accuracy = 0.929

EPOCH 386 ...
Validation Accuracy = 0.931

EPOCH 387 ...
Validation Accuracy = 0.920

EPOCH 388 ...
Validation Accuracy = 0.922

EPOCH 389 ...
Validation Accuracy = 0.917

EPOCH 390 ...
Validation Accuracy = 0.937

EPOCH 391 ...
Validation Accuracy = 0.925

EPOCH 392 ...
Validation Accuracy = 0.913

EPOCH 393 ...
Validation Accuracy = 0.926

EPOCH 394 ...
Validation Accuracy = 0.932

EPOCH 395 ...
Validation Accuracy = 0.926

EPOCH 396 ...
Validation Accuracy = 0.927

EPOCH 397 ...
Validation Accuracy = 0.917

EPOCH 398 ...
Validation Accuracy = 0.933

EPOCH 399 ...
Validation Accuracy = 0.924

EPOCH 400 ...
Validation Accuracy = 0.932

Training Accuracy = 0.993
Model saved


---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [3]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [4]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [3]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")